<a href="https://colab.research.google.com/github/yardsale8/probability_simulations_in_R/blob/main/2_5_introduction_to_parametric_simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(tidyverse)
library(devtools)
install_github('yardsale8/purrrfect', force = TRUE)
library(purrrfect)

# Introduction to Parametric Simulations

This chapter we have been studying various discrete parametric distributions.  Sometimes we wish to explore the properties of such a distribution over a set of parameters, and in this case, we can use the `tidyverse` and `purrr` toolset to capture the results.

## Outline

1. Setting up the parameter space using `tribble`.
2. Unnesting the parameters and adding trials.
3. Mapping over the parameters using `pmap`
4. Composing operations to save memory on large simulations.
5. Parametric sampling.

## First Motivating Problem - One Variable Parameter

<font color="aqua">**Our Task.**.   </font> Suppose we wish to explore the effects of the sample size on the mean and variance of a binomial random variable when $p=0.5$.  

<font color="aqua">**The Problem.** </font> Our current approach would require a separate simulation/pipe for each sample size.

<font color="aqua">**The Solution.**</font> Store the sample sizes in our experimental notebook and use them as mapping inputs.

### Three Approachs

1. Stack the trials into a long table, transform, and then group and aggregate. <font size="1">(optional, covered at the end of the notebook)</font>.
2. Store the trials in a list column, then `map` transformations/aggregations.
3. Compose all actions from **2.** into a single `map`

## Performing a Parametric Simulation using a List Column

In this variation of the simulation, we will
1. Store all the trials for each parameter (or combination of parameters) in a single row.
2. Use `map` or `pmap` to process those trials.

### Defining the parameter space using a `parameters`

1. Define all the names on the first row preceeded by `~`
2. Define the respective collections on the second row, respectively.

In [2]:
parameters(~n,
         c(5,10,15))

n
<dbl>
5
10
15


### The "shape" of a parametric simulation using a list column.

1. Set up the parameter space using a `parameters`.
2. Use `map` or `pmap` to generate a list column of trials.
3. Use `map` or `pmap` to transform/summarize.
4. Drop the outcome column.

In [3]:
# Set up parameter space
(parameters(~n,
          c(5,10,15))
 )

n
<dbl>
5
10
15


In [4]:
# Use map to generate trials
p <- 0.5
num.trials <- 10
(parameters(~n,
         c(5,10,15))
 %>% mutate(.outcome = map(n, \(n) rbinom(num.trials, n, p)),
           )
 )

n,.outcome
<dbl>,<list>
5,"5, 2, 1, 4, 2, 2, 3, 2, 3, 3"
10,"4, 6, 3, 6, 8, 4, 4, 4, 7, 5"
15,"7, 7, 8, 7, 5, 7, 9, 10, 9, 8"


In [5]:
# Transform/summarize
p <- 0.5
num.trials <- 10
(parameters(~n,
         c(5,10,15))
 %>% mutate(.outcome = map(n, \(n) rbinom(num.trials, n, p)),
            approx.mu = map_dbl(.outcome, mean),
            exact.mu = n*p
           )
 )

n,.outcome,approx.mu,exact.mu
<dbl>,<list>,<dbl>,<dbl>
5,"3, 3, 2, 3, 3, 4, 2, 3, 3, 2",2.8,2.5
10,"4, 4, 5, 9, 8, 7, 4, 6, 6, 6",5.9,5.0
15,"9, 9, 6, 9, 12, 7, 10, 11, 6, 5",8.4,7.5


In [6]:
# Drop outcomes
p <- 0.5
num.trials <- 10
(parameters(~n,
            c(5,10,15))
 %>% mutate(.outcome = map(n, \(n) rbinom(num.trials, n, 0.5)),
            approx.mu = map_dbl(.outcome, mean),
            exact.mu = n*p
           )
 %>% select(-.outcome)
 )

n,approx.mu,exact.mu
<dbl>,<dbl>,<dbl>
5,2.2,2.5
10,5.3,5.0
15,7.0,7.5


In [7]:
# Good estimate by bumping the num.trials
p <- 0.5
num.trials <- 100000
(parameters(~n,
          c(5,10,15))
 %>% mutate(.outcome = map(n, \(n) rbinom(num.trials, n, 0.5)),
            approx.mu = map_dbl(.outcome, mean),
            exact.mu = n*p
           )
 %>% select(-.outcome)
 )

n,approx.mu,exact.mu
<dbl>,<dbl>,<dbl>
5,2.50118,2.5
10,4.99632,5.0
15,7.50122,7.5


### The Advantage to using a list column of outcomes

The advantage of storing the trials in a list column is
1. All the information is self-contained and apparent, and
2. Makes it easier to verify your code and debug.

In [8]:
# Transform/summarize
p <- 0.5
num.trials <- 10
(parameters(~n,
            c(5,10,15))
 %>% mutate(.outcome = map(n, \(n) rbinom(num.trials, n, 0.5)), # Easy to
            approx.mu = map_dbl(.outcome, mean),                # verify
            exact.mu = n*p                                      # correctness
           )
 )

n,.outcome,approx.mu,exact.mu
<dbl>,<list>,<dbl>,<dbl>
5,"1, 3, 4, 3, 2, 3, 4, 1, 3, 3",2.7,2.5
10,"5, 5, 5, 6, 6, 8, 4, 5, 3, 7",5.4,5.0
15,"7, 5, 7, 5, 5, 7, 10, 8, 7, 10",7.1,7.5


### Two Problems with using a list column of outcomes

Two problems with storing outcomes in a list column are

#### 1. Displaying more than a few trials is a mess

In [9]:
# Slightly Better estimates
num.trials <- 100
(parameters(~n,
            c(5,10,15))
 %>% mutate(.outcome = map(n, \(n) rbinom(num.trials, n, 0.5)), # Yuck! So long!
            approx.mu = map_dbl(.outcome, mean),
            exact.mu = n*p
           )
 )

n,.outcome,approx.mu,exact.mu
<dbl>,<list>,<dbl>,<dbl>
5,"1, 1, 3, 3, 1, 4, 3, 4, 3, 1, 1, 3, 4, 4, 3, 3, 3, 2, 2, 3, 2, 2, 4, 5, 2, 2, 4, 3, 3, 3, 3, 3, 3, 4, 2, 3, 1, 3, 2, 3, 4, 3, 2, 2, 3, 4, 1, 4, 0, 3, 3, 4, 2, 2, 2, 4, 2, 3, 4, 2, 4, 3, 5, 1, 3, 2, 1, 2, 0, 1, 2, 2, 1, 4, 1, 4, 2, 4, 3, 1, 3, 0, 3, 4, 5, 0, 2, 4, 2, 1, 2, 3, 2, 4, 4, 3, 0, 2, 3, 2",2.58,2.5
10,"3, 6, 4, 6, 5, 7, 6, 5, 3, 6, 5, 6, 6, 6, 5, 6, 3, 2, 3, 4, 5, 8, 4, 4, 8, 6, 8, 5, 5, 6, 8, 5, 5, 6, 4, 3, 4, 2, 4, 4, 5, 7, 3, 6, 6, 9, 4, 5, 6, 5, 5, 4, 8, 4, 4, 3, 5, 8, 3, 5, 6, 7, 8, 6, 7, 6, 5, 5, 5, 4, 4, 8, 6, 4, 8, 5, 5, 6, 4, 3, 6, 4, 4, 5, 6, 3, 4, 4, 4, 6, 6, 4, 6, 5, 2, 6, 4, 5, 3, 2",5.07,5.0
15,"9, 8, 5, 7, 9, 12, 5, 7, 6, 5, 7, 9, 5, 7, 9, 8, 6, 4, 7, 11, 9, 5, 6, 9, 12, 6, 9, 9, 8, 10, 5, 8, 6, 11, 9, 10, 7, 8, 5, 12, 7, 6, 7, 7, 12, 10, 5, 6, 10, 8, 8, 7, 11, 7, 5, 7, 9, 9, 8, 5, 9, 8, 8, 9, 4, 9, 8, 8, 7, 6, 7, 7, 10, 6, 9, 5, 5, 8, 9, 9, 7, 6, 5, 6, 6, 8, 9, 8, 9, 7, 8, 10, 9, 6, 6, 4, 9, 7, 6, 8",7.59,7.5


#### 2. We are storing a *lot* of data during most intermediate steps.

In [10]:
# Drop outcomes
num.trials <- 100000 # Start adding zeros and see what happens
(parameters(~n,
            c(5,10,15))
 %>% mutate(.outcome = map(n, \(n) rbinom(num.trials, n, 0.5)), # <== Lots of data/memory used
            approx.mu = map_dbl(.outcome, mean),
            exact.mu = n*p
           )
 %>% select(-.outcome)
 )

n,approx.mu,exact.mu
<dbl>,<dbl>,<dbl>
5,2.50171,2.5
10,5.00667,5.0
15,7.50231,7.5


### The Solution

The solution is to

1. Prototype the code using a separate list column of outcomes,
2. Verify the correctness of your code, then
2. Compose the steps into a single function

## The "shape" of a composed parametric simulation

1. Set up the parameter space using a `parameters`.
2. Use one `map` or `pmap` to perform the entire process, usually by piping each intermediate result into the next function.

In [11]:
# First prototype WITH the outcome column
p <- 0.5
num.trials <- 10
(parameters(~n,
            c(5,10,15))
 %>% mutate(.outcome = map(n, \(n) rbinom(num.trials, n, 0.5)), # Easy to
            approx.mu = map_dbl(.outcome, mean),                # verify
            exact.mu = n*p                                      # correctness
           )
 )

n,.outcome,approx.mu,exact.mu
<dbl>,<list>,<dbl>,<dbl>
5,"4, 3, 2, 3, 3, 2, 1, 2, 3, 3",2.6,2.5
10,"5, 8, 9, 2, 6, 5, 6, 7, 8, 5",6.1,5.0
15,"7, 11, 8, 8, 7, 5, 9, 12, 7, 9",8.3,7.5


In [12]:
# Second ... Compose!
num.trials <- 100
(parameters(~n,
            c(5,10,15))
 %>% mutate(approx.mu = map_dbl(n, \(n) (rbinom(num.trials, n, 0.5)
                                        %>% mean)),
            exact.mu = n*p
           )
)

n,approx.mu,exact.mu
<dbl>,<dbl>,<dbl>
5,2.55,2.5
10,5.35,5.0
15,7.35,7.5


### When can you compose?

When each step is saved, then immediately used on the next step.

#### Tracking the data

Here the raw trials are saved as `.outcome`, which is then used as input on the next mutate.

In [13]:
num.trials <- 10
(parameters(~n,
            c(5,10,15))# ───────┐
 %>% mutate(.outcome = map(n, \(n) rbinom(num.trials, n, 0.5)),
            # │  └─────────────────────┘
            # └────────────────────┐
            approx.mu = map_dbl(.outcome, mean),
            #   │                  └──────┘ │
            #   └───────────────────────────┘
            exact.mu = n*p
           )
 %>% select(-.outcome)
 )

n,approx.mu,exact.mu
<dbl>,<dbl>,<dbl>
5,2.6,2.5
10,4.1,5.0
15,7.0,7.5


#### Tracking the data

Instead of saving the intermediate result (i.e., the raw output) we can simply pipe it into the next action.

In [14]:
# Compose!
num.trials <- 100
(parameters(~n,
            c(5,10,15))# ────────────┐
 %>% mutate(approx.mu = map_dbl(n, \(n) (rbinom(num.trials, n, 0.5)
          #   │                              │
                                        %>% mean)),
          #   └──────────────────────────────┘
            exact.mu = n*p)
           )


n,approx.mu,exact.mu
<dbl>,<dbl>,<dbl>
5,2.40,2.5
10,4.96,5.0
15,7.79,7.5


### <font color="red"> Exercise 2.5.1 </font>

Suppose that we want to compare the cut off for the top 5% of a binomial data with $n = 100$ and $p\in\{0.25, 0.5, 0.75\}

**Tasks.**
1. Prototype the process by storing the raw outcomes in a list column, then processing with additional `map`s,
2. Verify the correctness of your code, and
2. Compose all the intermediate steps to eliminate the need to store the raw outcomes.

In [ ]:
# Your code here.

## Parametric Simulations with more than one parameter

A similar process can be used to simulate a scenario where two or more parameters vary across the experiment.  Depending on the scenario, we will
1. **Two parameters.** Use either `map2` or `pmap` to generate the raw outcomes.
2. **Three+ parameters.** Use `pmap` to generate the raw outcomes.

### Example 2 - Investigate the mean part 2

Now suppose we want to investigate the mean of the binomial distribution for all combinations of $n\in\{5,10,15\}$ and $p\in\{0.25, 0.5, 0.74\}$.  

#### Step 1.  Define the parameter space

In [15]:
# Define the parameter space
parameters(~n,         ~p,
           c(5,10,15), c(0.25, 0.5, 0.75))

n,p
<dbl>,<dbl>
5,0.25
5,0.50
5,0.75
10,0.25
10,0.50
10,0.75
15,0.25
15,0.50
15,0.75


#### Step 2.  Generate outcomes with `map2`

In [16]:
# Version 1 - Use map2 to generate the data
num.trials <- 10
(parameters(~n,         ~p,
            c(5,10,15), c(0.25, 0.5, 0.75))
 %>% mutate(.outcome = map2(n, p, \(n, p) rbinom(num.trials, n, p)))
 )

n,p,.outcome
<dbl>,<dbl>,<list>
5,0.25,"2, 2, 0, 2, 0, 2, 3, 0, 0, 1"
5,0.50,"2, 2, 1, 2, 0, 1, 3, 5, 3, 1"
5,0.75,"4, 3, 5, 4, 4, 4, 3, 5, 3, 3"
10,0.25,"1, 3, 2, 2, 3, 2, 2, 2, 2, 1"
10,0.50,"6, 7, 8, 8, 1, 6, 2, 4, 7, 6"
10,0.75,"7, 4, 6, 9, 7, 10, 8, 8, 6, 9"
15,0.25,"5, 5, 2, 4, 6, 3, 6, 6, 2, 4"
15,0.50,"6, 8, 7, 5, 6, 5, 7, 9, 7, 11"
15,0.75,"10, 13, 8, 7, 14, 13, 12, 13, 10, 13"


#### Step 2. Generate outcomes with `pmap`.

In [17]:
# Version 2 - Use pmap to generate the data
num.trials <- 10
(parameters(~n,         ~p,
            c(5,10,15), c(0.25, 0.5, 0.75))
 %>% mutate(.outcome = pmap(list(size = n, prob = p), rbinom, n = num.trials))
 )

n,p,.outcome
<dbl>,<dbl>,<list>
5,0.25,"0, 1, 1, 1, 0, 1, 0, 2, 3, 0"
5,0.50,"4, 3, 0, 2, 2, 4, 1, 3, 2, 4"
5,0.75,"4, 2, 4, 4, 5, 5, 3, 5, 3, 2"
10,0.25,"2, 3, 1, 2, 2, 2, 1, 4, 2, 5"
10,0.50,"7, 5, 5, 6, 3, 5, 6, 7, 4, 7"
10,0.75,"9, 6, 8, 9, 7, 9, 6, 8, 7, 7"
15,0.25,"5, 6, 6, 5, 6, 5, 8, 7, 4, 4"
15,0.50,"9, 10, 10, 6, 10, 6, 9, 11, 7, 6"
15,0.75,"10, 13, 14, 10, 11, 10, 11, 15, 12, 10"


#### Step 3. Process and summarize.

In [19]:
num.trials <- 10
(parameters(~n,         ~p,
            c(5,10,15), c(0.25, 0.5, 0.75))
 %>% mutate(.outcome = map2(n, p, \(n, p) rbinom(num.trials, n, p)))
 %>% mutate(approx.mu = map_dbl(.outcome, mean),
            exact.mu = n*p)
 )

n,p,.outcome,approx.mu,exact.mu
<dbl>,<dbl>,<list>,<dbl>,<dbl>
5,0.25,"1, 1, 1, 0, 1, 1, 2, 1, 0, 0",0.8,1.25
5,0.50,"3, 3, 2, 4, 5, 3, 2, 4, 2, 1",2.9,2.50
5,0.75,"1, 4, 3, 4, 3, 4, 4, 4, 5, 5",3.7,3.75
10,0.25,"0, 2, 2, 2, 3, 2, 2, 0, 3, 1",1.7,2.50
10,0.50,"6, 6, 3, 5, 4, 3, 6, 1, 4, 7",4.5,5.00
10,0.75,"7, 9, 8, 8, 8, 9, 9, 6, 8, 6",7.8,7.50
15,0.25,"4, 2, 2, 1, 5, 6, 5, 4, 3, 9",4.1,3.75
15,0.50,"8, 9, 6, 7, 10, 8, 6, 9, 4, 8",7.5,7.50
15,0.75,"13, 11, 9, 11, 4, 8, 13, 12, 12, 10",10.3,11.25


#### Step 5. Drop outcomes and bump the number of trials.

In [20]:
num.trials <- 100000
(parameters(~n,         ~p,
            c(5,10,15), c(0.25, 0.5, 0.75))
 %>% mutate(.outcome = map2(n, p, \(n, p) rbinom(num.trials, n, p)))
 %>% mutate(approx.mu = map_dbl(.outcome, mean),
            exact.mu = n*p)
 %>% select(-.outcome)
 )

n,p,approx.mu,exact.mu
<dbl>,<dbl>,<dbl>,<dbl>
5,0.25,1.24513,1.25
5,0.50,2.49446,2.50
5,0.75,3.75138,3.75
10,0.25,2.50023,2.50
10,0.50,5.00259,5.00
10,0.75,7.51119,7.50
15,0.25,3.75758,3.75
15,0.50,7.50161,7.50
15,0.75,11.24818,11.25


Step 5. Compose into one map.

In [2]:
num.trials <- 100000
(parameters(~n,         ~p,
            c(5,10,15), c(0.25, 0.5, 0.75))
#  %>% mutate(.outcome = map2(n, p, \(n, p) rbinom(num.trials, n, p))
#  %>% mutate(approx.mu = map_dbl(.outcome, mean),
            # exact.mu = n*p)
 %>% mutate(approx.mu = map2(n, p, \(n, p) rbinom(num.trials, n, p) %>% mean),
            exact.mu = n*p)
#  %>% select(-.outcome)
 )

n,p,approx.mu,exact.mu
<dbl>,<dbl>,<list>,<dbl>
5,0.25,1.24595,1.25
5,0.50,2.50038,2.50
5,0.75,3.74845,3.75
10,0.25,2.49804,2.50
10,0.50,4.99166,5.00
10,0.75,7.50035,7.50
15,0.25,3.74859,3.75
15,0.50,7.50473,7.50
15,0.75,11.25413,11.25


### <font color="red"> Exercise 2.5.2 </font>

Suppose that we want to compare the cut off for the top 5% of a binomial data with $n\in c\{25, 50, 100\}$ and $p\in\{0.25, 0.5, 0.75\}$

**Tasks.**
1. Prototype the process by storing the raw outcomes in a list column, then processing with additional `pmap`/`map2`s,
2. Verify the correctness of your code, and
2. Compose all the intermediate steps to eliminate the need to store the raw outcomes.

In [ ]:
# Your code here